# Adding Dependeces

In [1]:
using DelimitedFiles
using CUDA
using ProgressMeter
include("../../src/forces.jl")
include("../../src/Neighbor.jl")

knn_cu (generic function with 1 method)

# Initial Conditions

In [ ]:
# Running fusion of the two aggregates
# Physical Conditions
R_agg = 10

# Time model Conditions
T_f = 150000
dt = 0.1

# Neighbor Conditions
n_knn = 50
nn = 16

# Forces Conditions
R_Max = 3.5
S = 1.9
K_array = 0.005

# Running Fusion

In [ ]:
# Initial Data
X = Float32.(readdlm("../../../../../data/init/Sphere/$R_agg.xyz")[3:end,2:end]) |> cu

# Testing all the conditions of the fusion
for t_f in T_f
    for r_max in R_Max
        for s in S
            for K in K_array
                #Generating the path
                if !isdir("../results/T_$(t_f)") mkdir("T_$(t_f)") end
                if !isdir("../results/T_$(t_f)/rmax_$(r_max)_s_$(s)") mkdir("../results/T_$(t_f)/rmax_$(r_max)_s_$(s)") end
                if !isdir("../results/T_$(t_f)/rmax_$(r_max)_s_$(s)/k_$(K)") mkdir("../results/T_$(t_f)/rmax_$(r_max)_s_$(s)/k_$(K)")  end
                
                
                # Calculating all above
                if size(readdir("../results/T_$(T_f)/rmax_$(R_Max)_s_$(S)/k_$(K)"))[1] == 0
                    cd("../results/T_$(t_f)/rmax_$(r_max)_s_$(s)/k_$(K)")
                    println("Calculating T_Final=$(t_f) | R_Max = $(r_max) | s = $(s) | k = $(K)")
                    main(t_f, r_max, s, K)
                    X = Matrix(X)
                        open("Test_Final.xyz"; write=true) do f
                            write(f, "$(size(X, 1))\n")
                            write(f, "Initial Data ($(R_agg))\n")
                            writedlm(f,hcat(X_f_2, X), ' ')
                        end
                    cd("../../..")
                else
                    println("This is already calculated")
                end
            end
        end
    end
end

In [ ]:
# run(`paraview Test_Initial.xyz Test_Final.xyz`)